In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()
from agents import Agent, Runner,function_tool,trace, ItemHelpers, MessageOutputItem
import nest_asyncio
nest_asyncio.apply()
# setting up api key
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
import wikipedia
import openmeteo_requests
import requests_cache
# from retry_requests import retry
import requests
from agents.extensions.visualization import draw_graph

import json 
import requests
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")


from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

from tools import flight_search, yelp_search, hotels_search, wikipedia_search, get_weather
from pydantic import BaseModel
from models import first_agent_output
from agents.model_settings import ModelSettings
from data_models.flight_data_models import FlightSearchResults, BestFlight, AirportInfo, Flight, Layover, OtherFlight, PriceInsights, AirportLocation, Airports
from data_models.yelp_data_models import Category, Business, YelpSearchResults
from travel_agents import question_rewrite_agent, flight_agent
from functions import flight_search,yelp_search
from pprint import pprint
from data_models.first_question_output_format import FinalModel
import datetime

from tools import flight_search, yelp_search,hotels_search


In [2]:
ank = "ankara(ESB)"
ist = "istanbul(SAW)"
amst = "amsterdam(AMS)"
pekin = "Peking(PEK)"

start_point = ank 
dest_point = amst

main_question = f"""I want to travel from {start_point} to {dest_point}. The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
I also need a hotel to stay. I do not like smell of cigarette. If possible, I want to see the rooms where smoking is not allowed. 
In addition to that, I like eating sea food. I guess {dest_point} is very rich city interms of sea restaurants""".strip()

question_rewrite_agent_output = Runner.run_sync(question_rewrite_agent,main_question)

In [3]:
question_rewrite_agent_output.final_output.yelp_reqs

YelpOutput(search_term='sea food', location='Amsterdam')

In [4]:
x = yelp_search(
search_term = question_rewrite_agent_output.final_output.yelp_reqs.search_term, 
location = question_rewrite_agent_output.final_output.yelp_reqs.location 
)

In [ ]:
Category(**x[0])

Category(title='The Seafood Bar', link=HttpUrl('https://www.yelp.com/biz/the-seafood-bar-amsterdam-3?osq=sea+food'))

In [13]:
Business(**x[0]).price

'€€€'

In [23]:
yelp_search_res = ""
for i in range(len(x)):
    # print(x[i])
    yelp_search_res += f"\n{i+1}"
    yelp_search_res += f"""
Title : {Category(**x[i]).title}
Category : {Business(**x[i]).categories[0].title}
Rating : {Business(**x[i]).rating}
Neighborhoods : {Business(**x[i]).neighborhoods}
Price : {Business(**x[i]).price}
"""

In [24]:
print(yelp_search_res)


1
Title : The Seafood Bar
Category : Seafood
Rating : 4.5
Neighborhoods : Centrum
Price : €€€

2
Title : Pesca
Category : Seafood
Rating : 4.5
Neighborhoods : Jordaan
Price : €€

3
Title : The Seafood Bar
Category : Seafood
Rating : 4.1
Neighborhoods : Museumkwartier
Price : €€€

4
Title : The Seafood Shop
Category : Seafood
Rating : 4.2
Neighborhoods : Centrum
Price : None

5
Title : The Seafood Bar
Category : Seafood
Rating : 4.5
Neighborhoods : De Pijp
Price : €€€

6
Title : Frens Haringhandel
Category : Street Vendors
Rating : 4.5
Neighborhoods : Centrum
Price : €

7
Title : The Good Companion
Category : Fish & Chips
Rating : 4.8
Neighborhoods : Jordaan
Price : None

8
Title : Dutch Four
Category : Seafood
Rating : 5.0
Neighborhoods : Centrum
Price : None

9
Title : Black Steaks & Seafood
Category : Seafood
Rating : 4.4
Neighborhoods : Centrum
Price : None

10
Title : éénvistwéévis
Category : Seafood
Rating : 4.5
Neighborhoods : Centrum
Price : €€€



In [26]:
yelp_agent = Agent(
    name = "YELP SEARCH",
    instructions=f"You are helpful asisstants try to answer the questions according to: \n\n{yelp_search_res}. You are not allowed to recommend or answer any other thing than given information",
    model = 'gpt-4o-mini',
    # tools=[yelp_search],
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
)

result = Runner.run_sync(yelp_agent, main_question+" bring me cheapest restaurent")
print(result.final_output)

For your trip to Amsterdam, here are the details based on your preferences:

### Seafood Restaurant
- **Pesca**
  - **Category**: Seafood
  - **Rating**: 4.5
  - **Neighborhood**: Jordaan
  - **Price**: €€

This is the cheapest seafood restaurant option available.

### Hotel Recommendation
While I cannot provide specific hotel recommendations, I suggest looking for hotels in Amsterdam that explicitly state they are non-smoking. Many hotels offer non-smoking rooms, and you can filter your search to ensure you find accommodations that meet your preferences.

### Travel Dates
- **Outbound**: 2025-05-05
- **Return**: 2025-05-11

Make sure to book your flights and accommodations in advance to secure the best options!


In [30]:
yelp_agent = Agent(
    name = "YELP SEARCH",
    instructions=f"You are helpful asisstants try to answer the questions according to: \n\n{yelp_search_res}. You are not allowed to recommend or answer any other thing than given information",
    model = 'gpt-4o-mini',
    # tools=[yelp_search],
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
)

result = Runner.run_sync(yelp_agent, main_question+" bring me closest restaurants to the city center")
print(result.final_output)

Here are some seafood restaurants in Amsterdam that are close to the city center:

1. **The Seafood Bar**
   - **Rating:** 4.5
   - **Neighborhood:** Centrum
   - **Price:** €€€

2. **The Seafood Shop**
   - **Rating:** 4.2
   - **Neighborhood:** Centrum
   - **Price:** None

3. **Dutch Four**
   - **Rating:** 5.0
   - **Neighborhood:** Centrum
   - **Price:** None

4. **Black Steaks & Seafood**
   - **Rating:** 4.4
   - **Neighborhood:** Centrum
   - **Price:** None

5. **éénvistwéévis**
   - **Rating:** 4.5
   - **Neighborhood:** Centrum
   - **Price:** €€€

These options should provide you with a good selection of seafood dining experiences in the city center. 

For your hotel stay, I recommend looking for accommodations that specifically mention non-smoking rooms to ensure a comfortable experience.
